# Entity

In [2]:
import os

In [3]:
%pwd

'c:\\Users\\91981\\Desktop\\GREAT LEARNING\\FROM SCRATCH LEARNING\\Krish Naik Projects\\WIP\\chicken_disease\\research'

In [4]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.CD_Classifier.a_constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.CD_Classifier.f_utils.common import read_yaml, create_directories

In [8]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [9]:
read_yaml(CONFIG_FILE_PATH)

[ 2023-11-06 08:00:56,006 ] 31 root - INFO - yaml file: config\config.yaml loaded successfully


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}})

# Configuration

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [11]:
config_manager = ConfigurationManager()

[ 2023-11-06 08:01:13,135 ] 31 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2023-11-06 08:01:13,148 ] 31 root - INFO - yaml file: params.yaml loaded successfully
[ 2023-11-06 08:01:13,150 ] 50 root - INFO - created directory at: artifacts


In [12]:
data_ingestion_config = config_manager.get_data_ingestion_config()

[ 2023-11-06 08:01:22,217 ] 50 root - INFO - created directory at: artifacts/data_ingestion


In [13]:
data_ingestion_config.root_dir

'artifacts/data_ingestion'

In [14]:
import os
import sys
import urllib.request as request
from zipfile import ZipFile
from pathlib import Path

from CD_Classifier.b_entity.config_entity import DataIngestionConfig
from CD_Classifier import logging, CustomException
from CD_Classifier.f_utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):

        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logging.info(
                    f"{filename} download! with following info: \n{headers}")
            else:
                logging.info(
                    f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

        except Exception as e:
            logging.info(CustomException(e, sys))
            raise CustomException(e, sys)

    def extract_zip_file(self):

        try:

            unzip_dir = self.config.unzip_dir
            os.makedirs(unzip_dir, exist_ok=True)
            logging.info(f"{unzip_dir} Directory created")

            with ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_dir)

            logging.info(f"All files from zip were extracted to {unzip_dir}")

        except Exception as e:
            logging.info(CustomException(e, sys))
            raise CustomException(e, sys)

In [15]:
data_ingestion = DataIngestion(data_ingestion_config)

In [17]:
data_ingestion.config

DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')

In [18]:
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[ 2023-11-06 08:03:13,440 ] 24 root - INFO - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3226:9C1C9:6F06D:E256B:6548505E
Accept-Ranges: bytes
Date: Mon, 06 Nov 2023 02:33:04 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100035-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1699237984.865868,VS0,VE725
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: b6c4ce1bc010be551711f27623816db1c8fdd115
Expires: Mon, 06 Nov 2023 02:38:04 GMT
Source-Age: 0


[ 2023-11-06 08:03:13,440 ] 40 root - INFO 